In [2]:
import sys
import re
import numpy as np
import pandas as pd
import matplotlib as plt
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# np.set_printoptions(edgeitems=3)
# np.core.arrayprint._line_width = 80

In [3]:
path = 'data.txt'
data = open(path, 'r', encoding='utf-8').read().lower()
print('length of the corpus is: ', len(data))

length of the corpus is:  581888


In [4]:
# splitting the corpus into words

cleaned = re.sub(r'\W+', ' ', data).lower()
tokens = word_tokenize(cleaned)
print('Length of tokens: ' + str(len(tokens)))

# This includes all full stops and all the punctuation. We only need the words...
# tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
# words = tokenizer.tokenize(data)
# len(words)

Length of tokens: 109281


In [5]:
# Finding all the unique words

unique_words = np.unique(tokens)
len(unique_words)

8200

In [6]:
# Feature Engineering - converting the text into numerical values

train_len = 5
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)


In [7]:
text_sequences[0:10]

[['project', 'gutenberg', 's', 'the', 'adventures'],
 ['gutenberg', 's', 'the', 'adventures', 'of'],
 ['s', 'the', 'adventures', 'of', 'sherlock'],
 ['the', 'adventures', 'of', 'sherlock', 'holmes'],
 ['adventures', 'of', 'sherlock', 'holmes', 'by'],
 ['of', 'sherlock', 'holmes', 'by', 'arthur'],
 ['sherlock', 'holmes', 'by', 'arthur', 'conan'],
 ['holmes', 'by', 'arthur', 'conan', 'doyle'],
 ['by', 'arthur', 'conan', 'doyle', 'this'],
 ['arthur', 'conan', 'doyle', 'this', 'ebook']]

In [8]:
# Getting unique tokens

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
train_sequences = tokenizer.texts_to_sequences(text_sequences)
print('Found %s unique tokens.' % len(tokenizer.word_counts))
vocab =  len(tokenizer.word_counts)+1

Found 8200 unique tokens.


In [9]:
print(len(train_sequences))

109276


In [10]:
# creating an empty array

n_sequences = np.empty([len(train_sequences),train_len], dtype='int32')
for i in range(len(train_sequences)):
    n_sequences[i] = train_sequences[i]

x = n_sequences[:,:-1]
y = n_sequences[:,-1]

In [11]:
# Converting the input into binary

y = to_categorical(y,num_classes=vocab)

In [20]:
# Building the Model

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
model = Sequential()
model.add(Embedding(vocab,10, input_length=3))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocab, activation='relu'))

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 10)             82010     
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 50)             12200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 8201)              418251    
Total params: 535,211
Trainable params: 535,211
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 10)             82010     
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 50)             12200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 8201)              418251    
Total params: 535,211
Trainable params: 535,211
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model.fit(x, y,validation_split=0.05,batch_size=128, epochs=50)

Epoch 1/50
812/812 [==============================] - 93s 100ms/step - loss: 11.9724 - accuracy: 0.0485 - val_loss: 12.6458 - val_accuracy: 0.0560
Epoch 2/50
812/812 [==============================] - 38s 47ms/step - loss: 11.9487 - accuracy: 0.0566 - val_loss: 12.6170 - val_accuracy: 0.0635
Epoch 3/50
812/812 [==============================] - 46s 57ms/step - loss: 11.9113 - accuracy: 0.0681 - val_loss: 12.6571 - val_accuracy: 0.0703
Epoch 4/50
812/812 [==============================] - 40s 49ms/step - loss: 11.8799 - accuracy: 0.0772 - val_loss: 12.6076 - val_accuracy: 0.0695
Epoch 5/50
812/812 [==============================] - 43s 53ms/step - loss: 11.8609 - accuracy: 0.0821 - val_loss: 12.6144 - val_accuracy: 0.0739
Epoch 6/50
812/812 [==============================] - 39s 48ms/step - loss: 11.8762 - accuracy: 0.0838 - val_loss: 12.5742 - val_accuracy: 0.0677
Epoch 7/50
812/812 [==============================] - 40s 49ms/step - loss: 11.8336 - accuracy: 0.0905 - val_loss: 12.6113 

Epoch 50/50
812/812 [==============================] - 40s 50ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00


In [55]:
text = str(input("Enter the phrase: ")).strip().lower()

Enter the phrase: adventures of sherlock holmes


In [56]:
# Frunctions to predict the next word(s)

# # def predict(model,text):
# text = text.lower()
# encoded_text = tokenizer.texts_to_sequences([text])[0]
# # test_sequences = tokenizer.texts_to_sequences([text])
# pad_encoded = pad_sequences([encoded_text], maxlen=3, truncating='pre')
# for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
#     print(i)
#     pred_word = tokenizer.index_word[i]
#     print("Next word suggestion:",pred_word)

# predicted_word = model.predict(pad_encoded)
# label = predicted_word.argmax(axis=1)

# tokenizer.index_word[label[0]]

KeyError: 0

In [49]:
# predict(model, "adventures of sherlock holmes by")

KeyError: 0

In [36]:
# generate_text(model, "Harry took the invisibility", max_words=10)

In [59]:
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=3, truncating='pre')
# print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    pred_word = tokenizer.index_word[i]
    print("Next word suggestion:",pred_word)

adventures of sherlock
[983, 5, 125] [[983   5 125]]
8200
Next word suggestion: newsletter
2562
Next word suggestion: factor
2724
Next word suggestion: handy
